<a href="https://colab.research.google.com/github/hamasakram/Developers-Corporation-AI-Internship/blob/main/GoEmotionDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=pd.read_csv("go_emotions_dataset.csv")
df.head()

,id,text,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,eew5j0j,That game hurt.,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,eemcysk,>sexuality shouldn’t be a grouping category I...,True,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ed2mah1,"You do right, if you don't care then fuck 'em!",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,eeibobj,Man I love reddit.,False,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,eda6yn6,"[NAME] was nowhere near them, he was by the Fa...",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [2]:
!pip install transformers tensorflow scikit-learn pandas

**Preprocess the Dataset**

In [13]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

#removing 'id'column

# Filter out unclear examples (optional)
df = df[df['example_very_unclear'] == False]

# Separate features (text) and labels (emotions)
X = df['text']  # Input text
y = df.drop(columns=['text', 'example_very_unclear'])  # Emotion labels

# Split into training and validation sets
# Split into training and validation sets (using a subset of the data)
# Use only 50% of the data for training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, train_size=0.5)

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the text data
def tokenize_text(texts):
    return tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128, return_tensors="tf")

X_train_tokenized = tokenize_text(X_train)
X_val_tokenized = tokenize_text(X_val)

**Create Tensorflow Dataset**

In [14]:
import tensorflow as tf

# Convert labels to TensorFlow tensors
y_train_tensor = tf.convert_to_tensor(y_train.values, dtype=tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val.values, dtype=tf.float32)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_train_tokenized["input_ids"], "attention_mask": X_train_tokenized["attention_mask"]},
    y_train_tensor
)).batch(8).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_val_tokenized["input_ids"], "attention_mask": X_val_tokenized["attention_mask"]},
    y_val_tensor
)).batch(8).prefetch(tf.data.AUTOTUNE)

**Load and Compile Model**

In [15]:
from transformers import TFAutoModelForSequenceClassification

# Load a pre-trained model for multi-label classification
model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(y.columns)
)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  # Multi-label classification
    metrics=[tf.keras.metrics.BinaryAccuracy()]  # Add more metrics if needed
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

**Training the Model**

In [16]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

Epoch 1/3
12989/12989 [==============================] - 1752s 133ms/step - loss: 0.1274 - binary_accuracy: 0.9596 - val_loss: 0.1154 - val_binary_accuracy: 0.9606
Epoch 2/3
12989/12989 [==============================] - 1667s 128ms/step - loss: 0.1110 - binary_accuracy: 0.9615 - val_loss: 0.1148 - val_binary_accuracy: 0.9610
Epoch 3/3
12989/12989 [==============================] - 1664s 128ms/step - loss: 0.1027 - binary_accuracy: 0.9632 - val_loss: 0.1184 - val_binary_accuracy: 0.9608


In [17]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the validation set
y_pred = model.predict(val_dataset)
y_pred_binary = (y_pred.logits > 0).numpy()  # Apply threshold (e.g., 0.5)

# Generate a classification report
print(classification_report(y_val, y_pred_binary, target_names=y.columns))

5196/5196 [==============================] - 215s 41ms/step


AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [18]:
y_pred_binary = (y_pred.logits > 0)  # Apply threshold (e.g., 0.5)

# Generate a classification report
print(classification_report(y_val, y_pred_binary, target_names=y.columns))

                precision    recall  f1-score   support

    admiration       0.59      0.50      0.54      3429
     amusement       0.59      0.56      0.57      1785
         anger       0.51      0.20      0.28      1681
     annoyance       0.43      0.03      0.06      2767
      approval       0.49      0.09      0.15      3552
        caring       0.37      0.19      0.25      1189
     confusion       0.57      0.11      0.18      1494
     curiosity       0.47      0.26      0.34      1972
        desire       0.52      0.14      0.22       808
disappointment       0.55      0.04      0.07      1656
   disapproval       0.43      0.09      0.15      2323
       disgust       0.56      0.13      0.22      1095
 embarrassment       0.55      0.17      0.26       521
    excitement       0.42      0.12      0.19      1137
          fear       0.59      0.30      0.40       625
     gratitude       0.88      0.75      0.81      2323
         grief       0.00      0.00      0.00  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
# Save the model and tokenizer
model.save_pretrained("./emotion_detection_model")
tokenizer.save_pretrained("./emotion_detection_tokenizer")

('./emotion_detection_tokenizer/tokenizer_config.json',
 './emotion_detection_tokenizer/special_tokens_map.json',
 './emotion_detection_tokenizer/vocab.txt',
 './emotion_detection_tokenizer/added_tokens.json',
 './emotion_detection_tokenizer/tokenizer.json')

In [16]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import numpy as np

def load_model_and_tokenizer(model_path, tokenizer_path):
    """
    Load the saved model and tokenizer.
    """
    model = TFAutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    return model, tokenizer

def predict_emotion(text, model, tokenizer):
    """
    Predict emotions from the input text using the loaded model.
    """
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=128)

    # Predict using the model
    outputs = model(inputs)
    logits = outputs.logits

    # Apply a threshold (e.g., 0.5) to convert logits to binary predictions
    predictions = (logits > 0).numpy()

    return predictions

def get_emotion_labels(predictions, emotion_columns):
    """
    Map the binary predictions to emotion labels.
    """
    # Get the indices of the predicted emotions
    emotion_indices = np.where(predictions[0] == 1)[0]

    # Map indices to emotion labels
    predicted_emotions = [emotion_columns[i] for i in emotion_indices]

    return predicted_emotions

def main():
    # Paths to the saved model and tokenizer
    model_path = "./emotion_detection_model"
    tokenizer_path = "./emotion_detection_tokenizer"

    # Load the model and tokenizer
    model, tokenizer = load_model_and_tokenizer(model_path, tokenizer_path)

    # Example emotion columns (replace with your actual emotion labels)
    emotion_columns = [
        'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
        'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
        'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
        'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
    ]

    # Take user input
    user_input = input("Enter a text to detect emotions: ")

    # Predict emotions
    predictions = predict_emotion(user_input, model, tokenizer)

    # Get the predicted emotion labels
    predicted_emotions = get_emotion_labels(predictions, emotion_columns)

    # Display the results
    if predicted_emotions:
        print("Predicted Emotions:", ", ".join(predicted_emotions))
    else:
        print("No specific emotions detected.")



In [17]:
if __name__ == "__main__":
    main()

Some layers from the model checkpoint at ./emotion_detection_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_57']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./emotion_detection_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Enter a text to detect emotions: I am not in a mood to talk, get out!
Predicted Emotions: anger
